# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities = pd.read_csv("cities.csv", encoding="utf-8")

cities= cities.rename(columns={'Unnamed: 0': 'City_ID','Max Temp': 'max_temp'})

cities.head()

,City_ID,City,Lat,Lng,max_temp,Humidity,Cloudiness,Wind_Speed,Country,Date
0,0,albany,42.6001,-73.9662,48.00,77,100,3.00,US,1616806416
1,1,mataura,-46.1927,168.8643,64.99,70,100,5.99,NZ,1616806699
2,2,chuy,-33.6971,-53.4616,68.76,94,100,3.00,UY,1616806627
3,3,saryg-sep,51.5000,95.6000,16.68,85,14,2.24,RU,1616806699
4,4,lodja,-3.4833,23.4333,65.48,98,100,1.66,CD,1616806654


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)


In [4]:
lng_lat = cities[["Lat","Lng"]]
hum = cities['Humidity'].astype(float)

fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(lng_lat, weights=hum, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
cities_fil_df = cities.loc[(cities['Wind_Speed'] <= 10) & (cities['Cloudiness'] == 0) &
                          (cities['max_temp'] > 59) & (cities['max_temp'] <=80)].dropna()

cities_fil_df



,City_ID,City,Lat,Lng,max_temp,Humidity,Cloudiness,Wind_Speed,Country,Date
43,44,arica,-18.4750,-70.3042,68.00,73,0,0.74,CL,1616806711
65,67,perth,-31.9333,115.8333,69.01,59,0,3.44,AU,1616806778
81,83,arraial do cabo,-22.9661,-42.0278,78.80,83,0,6.91,BR,1616806783
127,132,abha,18.2164,42.5053,60.80,67,0,1.70,SA,1616806858
130,135,arlit,18.7369,7.3853,64.67,14,0,9.33,NE,1616806848
182,188,tabas,33.5959,56.9244,73.15,28,0,6.51,IR,1616806934
183,189,tura,25.5198,90.2201,72.36,16,0,2.42,IN,1616806934
215,221,barao de melgaco,-16.1944,-55.9675,78.80,78,0,3.44,BR,1616807003
231,237,ampanihy,-24.7000,44.7500,70.00,87,0,5.66,MG,1616807007
244,250,quelimane,-17.8786,36.8883,77.00,88,0,3.60,MZ,1616807071


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = cities_fil_df.loc[:,['City','Country','Lat','Lng']]

hotel_df.reset_index(inplace=True)


hotel_df["Hotel Name"] = ''

hotel_df.head()

,index,City,Country,Lat,Lng,Hotel Name
0,43,arica,CL,-18.4750,-70.3042,
1,65,perth,AU,-31.9333,115.8333,
2,81,arraial do cabo,BR,-22.9661,-42.0278,
3,127,abha,SA,18.2164,42.5053,
4,130,arlit,NE,18.7369,7.3853,


In [17]:

hotellist = []

for i in range(len(hotel_df)):
    lat = hotel_df.loc[i]['Lat']
    lng = hotel_df.loc[i]['Lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "keyword": "hotel",
        "key": g_key}
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    response = requested.json()
    try:
        hotellist.append(response['results'][0]['name'])
    except:
        hotellist.append("")
        
hotel_df["Hotel Name"] = hotellist
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.dropna(subset=['Hotel Name'], inplace=True)

hotel_df.head(50)

,index,City,Country,Lat,Lng,Hotel Name
0,43,arica,CL,-18.4750,-70.3042,Hotel Apacheta
1,65,perth,AU,-31.9333,115.8333,Quay Perth
2,81,arraial do cabo,BR,-22.9661,-42.0278,Orlanova Boutique Hotel
3,127,abha,SA,18.2164,42.5053,Shafa Abha Hotel
4,130,arlit,NE,18.7369,7.3853,Hôtel Temet
5,182,tabas,IR,33.5959,56.9244,Narenjestan Hotel
6,183,tura,IN,25.5198,90.2201,Hotel Polo Orchid
7,215,barao de melgaco,BR,-16.1944,-55.9675,Pousada Chez Barão Pantanal
8,231,ampanihy,MG,-24.7000,44.7500,Angora Hotel
9,244,quelimane,MZ,-17.8786,36.8883,RESIDENCIAL PÉROLA - Quelimane


In [18]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [20]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations)
fig.add_layer(markers)



# Display figure
fig

Figure(layout=FigureLayout(height='420px'))